In [1]:
from pyspark.sql import SparkSession

# Create a Spark session (local mode)
spark = (
    SparkSession.builder
    .appName("WeatherData2024_Load")
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
    .getOrCreate()
)

print("Spark session started")

25/10/13 23:29:05 WARN Utils: Your hostname, Astraeas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.224 instead (on interface en0)
25/10/13 23:29:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/13 23:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session started


25/10/13 23:29:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data_path = "../data/compacted/2024.parquet"
df = spark.read.parquet(data_path)
print("Data loaded successfully")
df.printSchema()

Data loaded successfully
root
 |-- STATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- WND: string (nullable = true)
 |-- CIG: string (nullable = true)
 |-- VIS: string (nullable = true)
 |-- TMP: string (nullable = true)
 |-- DEW: string (nullable = true)
 |-- SLP: string (nullable = true)



In [3]:
# Show a few records
df.show(20, truncate=False)

# Count total rows
print(f"Row count: {df.count():,}")

# Quick null check per column
for c in df.columns:
    nulls = df.filter(df[c].isNull()).count()
    print(f"{c:<10}: {nulls:,} missing")

+-----------+-------------------+----------+----------+---------+--------------+-----------+------------+-------+-------+-------+
|STATION    |DATE               |LATITUDE  |LONGITUDE |ELEVATION|WND           |CIG        |VIS         |TMP    |DEW    |SLP    |
+-----------+-------------------+----------+----------+---------+--------------+-----------+------------+-------+-------+-------+
|01001099999|2024-01-01T00:00:00|70.9333333|-8.6666667|9.0      |318,1,N,0061,1|99999,9,9,9|999999,9,9,9|-0070,1|-0130,1|10208,1|
|01001099999|2024-01-01T01:00:00|70.9333333|-8.6666667|9.0      |330,1,N,0051,1|99999,9,9,9|999999,9,9,9|-0065,1|-0124,1|10204,1|
|01001099999|2024-01-01T02:00:00|70.9333333|-8.6666667|9.0      |348,1,N,0035,1|99999,9,9,9|999999,9,9,9|-0065,1|-0113,1|10205,1|
|01001099999|2024-01-01T03:00:00|70.9333333|-8.6666667|9.0      |357,1,N,0019,1|99999,9,9,9|999999,9,9,9|-0064,1|-0105,1|10202,1|
|01001099999|2024-01-01T04:00:00|70.9333333|-8.6666667|9.0      |241,1,N,0008,1|99999,9,9,

In [4]:
#convert date column to timestamp and add year, month, day, hour columns
from pyspark.sql import functions as F

#Parse DATE col
df = df.withColumn("DATE", F.to_timestamp("DATE", "yyyy-MM-dd'T'HH:mm:ss"))

# Add new time cols
df = (
    df.withColumn("year",  F.year("DATE"))
      .withColumn("month", F.month("DATE"))
      .withColumn("day",   F.dayofmonth("DATE"))
      .withColumn("hour",  F.hour("DATE"))
)

# Reorder columns 
orig = df.columns
new_order = []
for c in orig:
    if c == "DATE":
        new_order += ["DATE", "year", "month", "day", "hour"]
    elif c not in {"year","month","day","hour"}:  # avoid duplicates
        new_order.append(c)

df = df.select(*new_order)

# sanity check
df.select("DATE", "year", "month", "day", "hour").show(5, truncate=False)
df.printSchema()

+-------------------+----+-----+---+----+
|DATE               |year|month|day|hour|
+-------------------+----+-----+---+----+
|2024-01-01 00:00:00|2024|1    |1  |0   |
|2024-01-01 01:00:00|2024|1    |1  |1   |
|2024-01-01 02:00:00|2024|1    |1  |2   |
|2024-01-01 03:00:00|2024|1    |1  |3   |
|2024-01-01 04:00:00|2024|1    |1  |4   |
+-------------------+----+-----+---+----+
only showing top 5 rows

root
 |-- STATION: string (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- WND: string (nullable = true)
 |-- CIG: string (nullable = true)
 |-- VIS: string (nullable = true)
 |-- TMP: string (nullable = true)
 |-- DEW: string (nullable = true)
 |-- SLP: string (nullable = true)



In [5]:
df.show(20, truncate=False)


+-----------+-------------------+----+-----+---+----+----------+----------+---------+--------------+-----------+------------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND           |CIG        |VIS         |TMP    |DEW    |SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+--------------+-----------+------------+-------+-------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318,1,N,0061,1|99999,9,9,9|999999,9,9,9|-0070,1|-0130,1|10208,1|
|01001099999|2024-01-01 01:00:00|2024|1    |1  |1   |70.9333333|-8.6666667|9.0      |330,1,N,0051,1|99999,9,9,9|999999,9,9,9|-0065,1|-0124,1|10204,1|
|01001099999|2024-01-01 02:00:00|2024|1    |1  |2   |70.9333333|-8.6666667|9.0      |348,1,N,0035,1|99999,9,9,9|999999,9,9,9|-0065,1|-0113,1|10205,1|
|01001099999|2024-01-01 03:00:00|2024|1    |1  |3   |70.9333333|-8.6666667|9.0      |357,1,N,0019,1|

In [6]:
from pyspark.sql import functions as F

def split_and_replace_safe(df, col_name: str, new_cols: list[str]):
    """
    Split a comma-separated column (e.g., 'WND') into multiple columns and
    replace it in-place without duplication or unresolved-column issues.
    """
    if col_name not in df.columns:
        return df  # no-op if column missing

    # Remember current order and insertion index
    cols_before = df.columns
    insert_idx = cols_before.index(col_name)

    # Ensure string and create a temp tokens array
    tmp_tokens = f"__{col_name}_tokens"
    df = df.withColumn(col_name, F.col(col_name).cast("string"))
    df = df.withColumn(tmp_tokens, F.split(F.col(col_name), ","))

    # Add new columns from the tokens 
    for i, nc in enumerate(new_cols):
        df = df.withColumn(nc, F.col(tmp_tokens).getItem(i))

    # Drop the original and the temp array
    df = df.drop(col_name, tmp_tokens)

    # Reorder columns so new cols sit exactly where the original was
    base_cols = [c for c in cols_before if c != col_name]
    final_order = base_cols[:insert_idx] + new_cols + base_cols[insert_idx:]
    df = df.select(*final_order)

    return df

In [7]:
df = split_and_replace_safe(df, "WND",
    ["WND_DIR_DEG","WND_DIR_QUAL","WND_TYPE","WND_SPD_RAW","WND_SPD_QUAL"])

df.select("STATION","DATE","WND_DIR_DEG","WND_DIR_QUAL","WND_TYPE","WND_SPD_RAW","WND_SPD_QUAL").show(10, truncate=False)
#df.printSchema()

+-----------+-------------------+-----------+------------+--------+-----------+------------+
|STATION    |DATE               |WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|
+-----------+-------------------+-----------+------------+--------+-----------+------------+
|01001099999|2024-01-01 00:00:00|318        |1           |N       |0061       |1           |
|01001099999|2024-01-01 01:00:00|330        |1           |N       |0051       |1           |
|01001099999|2024-01-01 02:00:00|348        |1           |N       |0035       |1           |
|01001099999|2024-01-01 03:00:00|357        |1           |N       |0019       |1           |
|01001099999|2024-01-01 04:00:00|241        |1           |N       |0008       |1           |
|01001099999|2024-01-01 05:00:00|076        |1           |N       |0048       |1           |
|01001099999|2024-01-01 06:00:00|084        |1           |N       |0054       |1           |
|01001099999|2024-01-01 07:00:00|040        |1           |N       |002

In [8]:
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+-----------+------------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG        |VIS         |TMP    |DEW    |SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+-----------+------------+-------+-------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1           |99999,9,9,9|999999,9,9,9|-0070,1|-0130,1|10208,1|
|01001099999|2024-01-01 01:00:00|2024|1    |1  |1   |70.9333333|-8.6666667|9.0      |330        |1           |N       |0051       |1           |99999,9,9,9|999999,9,9,9|-0065,1|-0124,1|10204,1|
|01001099999|2024-01-01 02:00:

In [9]:
df = split_and_replace_safe(df, "CIG",
    ["CIG_HEIGHT_M","CIG_QUAL","CIG_DET_CODE","CIG_CAVOK"])
df.show(10, truncate=False)


+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+------------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS         |TMP    |DEW    |SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+------------+-------+-------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1           |99999       |9       |9           |9        |999999,9,9,9|-0070,1|-0130,1|10208,1|
|01001099999|2024-01-01 01:00:00|2024|1    |1  |1   |70.9333333|-8.6666667|9.0      |330    

In [10]:
df = split_and_replace_safe(df, "VIS",
    ["VIS_DIST_M","VIS_QUAL","VIS_VAR_CODE","VIS_VAR_QUAL"])
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP    |DEW    |SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+-------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1           |99999       |9       |9           |9        |999999    |9       |9          

In [11]:
df = split_and_replace_safe(df, "TMP", ["TMP_RAW", "TMP_QUAL"])
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP_RAW|TMP_QUAL|DEW    |SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1           |99999       |9       |9           |9        |9999

25/10/13 23:29:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
# Split DEW ("-0130,1") → [dew_point_raw, quality_code]
df = split_and_replace_safe(df, "DEW", ["DEW_RAW", "DEW_QUAL"])
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP_RAW|TMP_QUAL|DEW_RAW|DEW_QUAL|SLP    |
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1           |99999       |9       |

In [13]:
# Split SLP ("10208,1") → [pressure_raw, quality_code]
df = split_and_replace_safe(df, "SLP", ["SLP_RAW", "SLP_QUAL"])
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP_RAW|TMP_QUAL|DEW_RAW|DEW_QUAL|SLP_RAW|SLP_QUAL|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|01001099999|2024-01-01 00:00:00|2024|1    |1  |0   |70.9333333|-8.6666667|9.0      |318        |1           |N       |0061       |1       

In [14]:
#count total rows and columns
print(f"Row count before cleansing: {df.count():,}, Column count: {len(df.columns)}")

Row count before cleansing: 130,222,106, Column count: 28


In [15]:
# remove rows with missing or invalid data based on data document
from pyspark.sql import functions as F

# WIND special cases:
# 999 direction = missing unless variable
missing_wind_dir = (F.col("WND_DIR_DEG") == "999") & (F.col("WND_TYPE") != "V")

# type '9' = missing unless speed == 0000
missing_wind_type = (F.col("WND_TYPE") == "9") & (F.col("WND_SPD_RAW") != "0000")

# define overall missing conditions for all weather elements
missing_conditions = (
    missing_wind_dir |
    missing_wind_type |
    (F.col("WND_SPD_RAW") == "9999") |
    (F.col("CIG_HEIGHT_M") == "99999") |
    #(F.col("CIG_DET_CODE") == "9") | #not removing because i don't think detemination code interfers with model
    #(F.col("CIG_CAVOK") == "9") | #not removing because i don't think CAVOK code interfers with model
    (F.col("VIS_DIST_M") == "999999") |
    (F.col("TMP_RAW").isin("+9999", "9999")) |
    (F.col("DEW_RAW").isin("+9999", "9999")) |
    (F.col("SLP_RAW") == "99999")
)

# Apply filter
df = df.filter(~missing_conditions)
print(f"Remaining rows after removing missing or invalid data: {df.count():,}")

Remaining rows after removing missing or invalid data: 18,825,732


In [16]:
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP_RAW|TMP_QUAL|DEW_RAW|DEW_QUAL|SLP_RAW|SLP_QUAL|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|01001099999|2024-01-01 12:00:00|2024|1    |1  |12  |70.9333333|-8.6666667|9.0      |069        |1           |N       |0068       |1       

In [17]:
# remove rows with suspect or erroneous quality codes
from pyspark.sql import functions as F

# accepted quality codes per variable
accepted_qc = {
    "WND_DIR_QUAL": ["0","1","4","5","9"],
    "WND_SPD_QUAL": ["0","1","4","5","9"],
    "CIG_QUAL":     ["0","1","4","5","9"],
    "VIS_QUAL":     ["0","1","4","5","9"],
    "VIS_VAR_QUAL": ["0","1","4","5","9"],
    "SLP_QUAL":     ["0","1","4","5","9"],
    "TMP_QUAL":     ["0","1","4","5","9","A","C","I","M","P","R","U"],
    "DEW_QUAL":     ["0","1","4","5","9","A","C","I","M","P","R","U"]
}

# Build condition: rows are valid only if each QC col is within allowed set
valid_condition = F.lit(True)
for col, good_vals in accepted_qc.items():
    valid_condition = valid_condition & (F.col(col).isin(good_vals))

# Filter
df = df.filter(valid_condition)
print(f"Remaining rows after removing suspect/erroneous quality codes: {df.count():,}")

Remaining rows after removing suspect/erroneous quality codes: 18,753,612


In [18]:
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_DIR_QUAL|WND_TYPE|WND_SPD_RAW|WND_SPD_QUAL|CIG_HEIGHT_M|CIG_QUAL|CIG_DET_CODE|CIG_CAVOK|VIS_DIST_M|VIS_QUAL|VIS_VAR_CODE|VIS_VAR_QUAL|TMP_RAW|TMP_QUAL|DEW_RAW|DEW_QUAL|SLP_RAW|SLP_QUAL|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+------------+--------+-----------+------------+------------+--------+------------+---------+----------+--------+------------+------------+-------+--------+-------+--------+-------+--------+
|01001099999|2024-01-01 12:00:00|2024|1    |1  |12  |70.9333333|-8.6666667|9.0      |069        |1           |N       |0068       |1       

In [19]:
#drop quality code columns as they are no longer needed
qc_cols = [
    "WND_DIR_QUAL","WND_SPD_QUAL",
    "CIG_QUAL", "CIG_DET_CODE", "CIG_CAVOK",
    "VIS_QUAL","VIS_VAR_QUAL", "VIS_VAR_CODE",
    "TMP_QUAL","DEW_QUAL",
    "SLP_QUAL"
]

df = df.drop(*qc_cols)

print(f"Dropped {len(qc_cols)} QC columns. New column count: {len(df.columns)}")

Dropped 11 QC columns. New column count: 17


In [20]:
df.show(20, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+-----------+------------+----------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_TYPE|WND_SPD_RAW|CIG_HEIGHT_M|VIS_DIST_M|TMP_RAW|DEW_RAW|SLP_RAW|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+-----------+------------+----------+-------+-------+-------+
|01001099999|2024-01-01 12:00:00|2024|1    |1  |12  |70.9333333|-8.6666667|9.0      |069        |N       |0068       |00600       |025000    |-0020  |-0052  |10172  |
|01001099999|2024-01-01 15:00:00|2024|1    |1  |15  |70.9333333|-8.6666667|9.0      |069        |N       |0080       |00600       |025000    |-0007  |-0038  |10152  |
|01001099999|2024-01-02 06:00:00|2024|1    |2  |6   |70.9333333|-8.6666667|9.0      |070        |N       |0059       |00300       |010000    |+0006  |-0004  |10072  

In [21]:
from pyspark.sql import functions as F

def scale_and_replace(df, col_name, expr):
    """Apply transformation and replace column at same position."""
    cols = df.columns
    idx = cols.index(col_name)
    df = df.withColumn(col_name, expr)
    # Reorder to keep column position consistent
    reordered = cols[:idx] + [col_name] + cols[idx+1:]
    return df.select(*reordered)

# WIND direction (1x)
df = scale_and_replace(df, "WND_DIR_DEG", F.col("WND_DIR_DEG").cast("double"))

# WIND speed (÷10)
df = scale_and_replace(df, "WND_SPD_RAW", (F.col("WND_SPD_RAW").cast("double") / 10.0))

# CEILING height (1x)
df = scale_and_replace(df, "CIG_HEIGHT_M", F.col("CIG_HEIGHT_M").cast("double"))

# VISIBILITY distance (1x)
df = scale_and_replace(df, "VIS_DIST_M", F.col("VIS_DIST_M").cast("double"))

# AIR temperature (÷10)
df = scale_and_replace(df, "TMP_RAW", (F.col("TMP_RAW").cast("double") / 10.0))

# DEW point (÷10)
df = scale_and_replace(df, "DEW_RAW", (F.col("DEW_RAW").cast("double") / 10.0))

# ATMOSPHERIC-PRESSURE pressure (÷10)
df = scale_and_replace(df, "SLP_RAW", (F.col("SLP_RAW").cast("double") / 10.0))

print("Scaled all numeric fields successfully!")
df.printSchema()


Scaled all numeric fields successfully!
root
 |-- STATION: string (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- WND_DIR_DEG: double (nullable = true)
 |-- WND_TYPE: string (nullable = true)
 |-- WND_SPD_RAW: double (nullable = true)
 |-- CIG_HEIGHT_M: double (nullable = true)
 |-- VIS_DIST_M: double (nullable = true)
 |-- TMP_RAW: double (nullable = true)
 |-- DEW_RAW: double (nullable = true)
 |-- SLP_RAW: double (nullable = true)



In [22]:
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+-----------+------------+----------+-------+-------+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_TYPE|WND_SPD_RAW|CIG_HEIGHT_M|VIS_DIST_M|TMP_RAW|DEW_RAW|SLP_RAW|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+-----------+------------+----------+-------+-------+-------+
|01001099999|2024-01-01 12:00:00|2024|1    |1  |12  |70.9333333|-8.6666667|9.0      |69.0       |N       |6.8        |600.0       |25000.0   |-2.0   |-5.2   |1017.2 |
|01001099999|2024-01-01 15:00:00|2024|1    |1  |15  |70.9333333|-8.6666667|9.0      |69.0       |N       |8.0        |600.0       |25000.0   |-0.7   |-3.8   |1015.2 |
|01001099999|2024-01-02 06:00:00|2024|1    |2  |6   |70.9333333|-8.6666667|9.0      |70.0       |N       |5.9        |300.0       |10000.0   |0.6    |-0.4   |1007.2 

In [23]:
df = df.withColumnRenamed("WND_SPD_RAW", "WND_SPD_MS") \
       .withColumnRenamed("TMP_RAW", "TMP_C") \
       .withColumnRenamed("DEW_RAW", "DEW_C") \
       .withColumnRenamed("SLP_RAW", "SLP_hPa")

In [24]:
df.show(10, truncate=False)

+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+----------+------------+----------+-----+-----+-------+
|STATION    |DATE               |year|month|day|hour|LATITUDE  |LONGITUDE |ELEVATION|WND_DIR_DEG|WND_TYPE|WND_SPD_MS|CIG_HEIGHT_M|VIS_DIST_M|TMP_C|DEW_C|SLP_hPa|
+-----------+-------------------+----+-----+---+----+----------+----------+---------+-----------+--------+----------+------------+----------+-----+-----+-------+
|01001099999|2024-01-01 12:00:00|2024|1    |1  |12  |70.9333333|-8.6666667|9.0      |69.0       |N       |6.8       |600.0       |25000.0   |-2.0 |-5.2 |1017.2 |
|01001099999|2024-01-01 15:00:00|2024|1    |1  |15  |70.9333333|-8.6666667|9.0      |69.0       |N       |8.0       |600.0       |25000.0   |-0.7 |-3.8 |1015.2 |
|01001099999|2024-01-02 06:00:00|2024|1    |2  |6   |70.9333333|-8.6666667|9.0      |70.0       |N       |5.9       |300.0       |10000.0   |0.6  |-0.4 |1007.2 |
|01001099999|2024-01-02 09:0

In [25]:
from pyspark.sql import functions as F

def replace_in_place(df, col_name, expr):
    cols = df.columns
    idx = cols.index(col_name)
    df = df.withColumn(col_name, expr)
    return df.select(*(cols[:idx] + [col_name] + cols[idx+1:]))

if "LATITUDE" in df.columns:
    df = replace_in_place(df, "LATITUDE",  F.col("LATITUDE").cast("double"))
if "LONGITUDE" in df.columns:
    df = replace_in_place(df, "LONGITUDE", F.col("LONGITUDE").cast("double"))
if "ELEVATION" in df.columns:
    df = replace_in_place(df, "ELEVATION", F.col("ELEVATION").cast("double"))
if "STATION" in df.columns:
    df = replace_in_place(df, "STATION", F.col("STATION").cast("integer"))
df.printSchema()

root
 |-- STATION: integer (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- WND_DIR_DEG: double (nullable = true)
 |-- WND_TYPE: string (nullable = true)
 |-- WND_SPD_MS: double (nullable = true)
 |-- CIG_HEIGHT_M: double (nullable = true)
 |-- VIS_DIST_M: double (nullable = true)
 |-- TMP_C: double (nullable = true)
 |-- DEW_C: double (nullable = true)
 |-- SLP_hPa: double (nullable = true)



In [26]:
#output cleansed data to new parquet file
output_path = "../data/cleansed/2024_cleansed.parquet"

# Coalesce(1) ensures a single output file
df.coalesce(1).write.mode("overwrite").parquet(output_path)

print(f"Cleansed dataset written to: {output_path}")

Cleansed dataset written to: ../data/cleansed/2024_cleansed.parquet


In [27]:
#split cleansed data into 70% training and 30% data parquet
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

train_output = "../data/train_2024.parquet"
test_output  = "../data/test_2024.parquet"

(train_df
    .coalesce(1)  # merge all partitions into one file
    .write.mode("overwrite")
    .parquet(train_output))

(test_df
    .coalesce(1)
    .write.mode("overwrite")
    .parquet(test_output))